# nbd

> Extra command line utilities for nbdev

In [ ]:
#| default_exp nbd

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
from fastcore.all import *
import itertools as it
import os, time
from ghapi.all import *

import configparser
from pathlib import Path

from oztools.core import *
from oztools.gh import *

import subprocess

import json
from fastcore.net import HTTP422UnprocessableEntityError

from glob import glob
import re, yaml, shutil
import importlib.resources as res

import asyncio
from asyncinotify import Inotify, Event, Mask
from pathlib import Path
from datetime import datetime

In [ ]:
path = Path("../..")
Path.BASE_PATH = path

In [ ]:
#| export
def make_things_pretty():
    # Fix not being able to click on "source" link in docs
    with open("./nbs/styles.css", 'a') as f: f.write("\nh3 {\n  width: fit-content;\n}")
    # Add dark theme and make bright theme compatible with in in colorscheme
    theme = {'light': 'united', 'dark': 'superhero'}
    with open("./nbs/_quarto.yml", 'r') as f: data = yaml.safe_load(f)
    data['format']['html']['theme'] = theme
    data['website']['favicon'] = 'favicon.png'
    with open("./nbs/_quarto.yml", 'w') as f: f.write(yaml.dump(data))
    # Copy favicon
    with res.as_file(res.files("oztools")/"data/favicon.png") as f: shutil.copy(f, "./nbs")

In [ ]:
#| export

def nbd_new_fn(name:str, description:str, license:str="Apache-2.0", private:bool=False):
    "Create a new nbdev project and setup github repo for it"
    gh_repo, local_repo = gh_new_repo_fn(name, description, license, private)
    # this one makes github run CI twice (not nice)
    #setup_pages_branch(local_repo, gh_repo.name)
    os.chdir(gh_repo.name)
    subprocess.run(["nbdev_new"]) # TODO: use .__wrapped__ property to extract original function
    subprocess.run(["nbdev_install_hooks"])
    make_things_pretty()
    subprocess.run(["nbdev_prepare"])
    subprocess.run(["nbdev_clean"])
    # Hopefully using pip + some sleep would be enough for github to be ready
    # to enable pages branch
    commit_and_push(local_repo, "Initial commit")
    subprocess.run(["pip", "install", "-e", ".[dev]"])
    print("Waiting for the github to finish build before enabling pages")
    print("Feel free to start working on the project")
    # TODO: maybe check github api if build is ready
    while True:
        time.sleep(20)
        try:
            setup_pages_branch_location(local_repo, gh_repo.name)
            break
        except HTTP422UnprocessableEntityError:
            pass

In [ ]:
#| export
@call_parse
def nbd_new(name:str, description:str, license:str="Apache-2.0", private:bool=False):
    "Create a new nbdev project and setup github repo for it"
    nbd_new_fn(name, description, license, private)

In [ ]:
#| export

def new_notebook_template(name, description):
    template = {
        'cells': [
            {'cell_type': 'markdown', 'metadata': {},
             'source': [
                    f'# {name}\n',
                    '\n',
                    f'> {description}'
                ]},
            {'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [],
             'source': [f'#| default_exp {name}']},
            {'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [],
             'source': [
                 '#| hide\n',
                 'from nbdev.showdoc import *'
             ]},
            {'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [],
             'source': [
                 '#| export\n',
                 'from fastcore.all import *'
             ]},
            {'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [],
             'source': [
                 '#| hide\n',
                 'import nbdev; nbdev.nbdev_e'+'xport()' # nbdev_e**ort is a forbidden word
             ]},
      ], 'metadata': { 'kernelspec': { 'display_name': 'python3', 'language': 'python', 'name': 'python3' } }, 'nbformat': 4, 'nbformat_minor': 4
    }
    return template

FIXME: Cell below results in an error for some reason

In [ ]:
template = new_notebook_template("foo", "Makes foo using bar")
print('\n---\n\n'.join(L(template['cells']).attrgot('source').map(''.join)))

# foo

> Makes foo using bar
---

#| default_exp foo
---

#| hide
from nbdev.showdoc import *
---

#| export
from fastcore.all import *
---

#| hide
import nbdev; nbdev.nbdev_export()


In [ ]:
m = re.match(r'(\d+).*\.ipynb', '99ab_asd.ipynb')
m.group(1)

'99'

In [ ]:
#| export
def zero_pad(num):
    num = str(num)
    return num if len(num) > 1 else f"0{num}"

In [ ]:
test_eq(zero_pad(9), '09')
test_eq(zero_pad(32), '32')

In [ ]:
#| export
@call_parse
def nbd_add(name:str, description:str,
            at:Optional[int] = None # If specified, insert new notebook at a specific position
           ):
    "Add new notebook to the project"

    prev_nbs = L(glob("[0-9]*?*.ipynb"))
    prev_ids = prev_nbs.map(lambda x: int(re.match(r'(\d+).*\.ipynb', x).group(1)))
    prev_id = max(prev_ids)

    new_id = prev_id + 1
    new_id = zero_pad(new_id)
    template = new_notebook_template(name, description)

    with open(f"{new_id}_{name}.ipynb", 'w') as f:
        json.dump(template, f)

## nbd_watch

Adapted from https://github.com/ProCern/asyncinotify/blob/master/examples/recursivewatch.py

In [ ]:
#| export
def get_directories_recursive(path: Path):
    return (Path(d) for d,_,_ in os.walk(path))

In [ ]:
list(get_directories_recursive(Path('.')))

[Path('.'),
 Path('.ipynb_checkpoints'),
 Path('02_nbd'),
 Path('02_nbd/favicons'),
 Path('02_nbd/favicons/arlantr'),
 Path('02_nbd/favicons/food-ocal'),
 Path('02_nbd/favicons/food-ocal/vegetable'),
 Path('02_nbd/favicons/food-ocal/treat'),
 Path('02_nbd/favicons/food-ocal/pasta'),
 Path('02_nbd/favicons/food-ocal/fruit'),
 Path('02_nbd/favicons/food-ocal/drink'),
 Path('02_nbd/favicons/food-ocal/dairy'),
 Path('02_nbd/favicons/food-ocal/meat')]

Algorithm:
1. Watch forever until an event happens.
2. After that, watch for 1s. If no more events fire, yield an event. Go back to step 1.
3. If more events fire in that 1s, record those events and go back to step 2.

Extra features:
1. If new directories are added, mark them for watching.
2. Skip events we are not interested in

In [ ]:
#| export
def watch_new_directories(inotify, event):
    if Mask.CREATE in event.mask and event.path is not None and event.path.is_dir():
        for directory in get_directories_recursive(event.path): inotify.add_watch(directory, mask)

In [ ]:
#| export
def _inotify_watch(inotify, path, debounce_interval):
    mask = Mask.CREATE | Mask.MODIFY
    for directory in get_directories_recursive(path): inotify.add_watch(directory, mask)

    combined_event = []
    def process_event(event):
        watch_new_directories(inotify, event)
        if not event.mask & mask: return False
        combined_event.append(event)
        return True

    while True:
        inotify.sync_timeout=-1 # Watch forever
        for event in inotify:
            if not process_event(event): continue

            inotify.sync_timeout = debounce_interval
            for event_seq in inotify:
                if not process_event(event): continue

            # No more events seen in sync_timeout: yield events seen in this event sequence
            if combined_event:
                yield combined_event
                combined_event = []

In [ ]:
#| export
def inotify_watch(path, debounce_interval=0.5):
    with Inotify() as inotify:
        yield from _inotify_watch(inotify, path, debounce_interval)

In [ ]:
#| export
def nbd_watch():
    "Watch `nbs` folder and automatically run `nbdev_export` on file change"
    for el in inotify_watch(Path('nbs')):
        print(f"[{datetime.now()}] {el[0].name}")
        subprocess.run(["nbdev_export"])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()